In [20]:
import pandas as pd
import time
import numpy as np
import math
import matplotlib.pyplot as plt
# Torch package
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch
from torch import nn
from torch.utils.data import Dataset
import pennylane as qml
import torch.optim as optim
import torch.utils.data as data
import random
import tensorflow as tf

In [21]:
tf.random.set_seed(15)
torch.manual_seed(42)
random.seed(42)
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams.update({'font.size':20})
legend_prop = {'weight':'bold'}
from pylab import rcParams
rcParams['axes.linewidth'] = 2

# Data

In [22]:
df = pd.read_csv('Processed_data_till_1_march.csv')
df = df.drop([ 'Unnamed: 0'], axis=1)
# Utilize data from Sep 2011 till 1st March 2024
df=df[50:]
# Remove few redundant data
df=df.drop(columns=['Date'])
df=df.drop(columns=['CUMLOGRET_1'])
df=df.drop(columns=['Gold in USD volume'])
df=df.drop(columns=['Open'])
df=df.drop(columns=['High'])
df=df.drop(columns=['Low'])

In [23]:
# Set target and features
target = "Close"
features = list(df.columns.difference(["Close"]))
print(features)
print(len(features))

['3M', 'BBB_20_2.0', 'BBL_20_2.0', 'BBM_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'Close_copy', 'Crude Futures_close', 'Crude Futures_volume', 'Crude_H-L', 'Crude_O-C', 'EMA_14', 'EMA_21', 'EMA_7', 'FTSE_H-L', 'FTSE_O-C', 'GBP USD ', 'GBP_USD_H-L', 'GBP_USD_O-C', 'Gold in USD close', 'Gold_H-L', 'Gold_O-C', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'SMA_14', 'SMA_21', 'SMA_7', 'Volume']
29


In [24]:
target_mean = df[target].mean()
target_stdev = df[target].std()

mean=dict()
stdev=dict()
for c in df.columns:
    mean[c] = df[c].mean()
    stdev[c] = df[c].std()

    df[c] = (df[c] - mean[c]) / stdev[c]

In [6]:
df.head()

,Close,Volume,EMA_7,EMA_14,EMA_21,SMA_7,SMA_14,SMA_21,MACD_12_26_9,MACDh_12_26_9,...,3M,FTSE_H-L,FTSE_O-C,GBP_USD_H-L,GBP_USD_O-C,Gold_H-L,Gold_O-C,Crude_H-L,Crude_O-C,Close_copy
50,-2.216035,0.076483,-2.142262,-2.152261,-2.194267,-2.125236,-2.076858,-2.136731,0.580627,-0.832570,...,-0.051602,1.239983,0.263215,0.069296,-0.504898,-0.245371,0.621244,0.493902,0.076677,-2.216035
51,-2.056965,-0.239245,-2.123602,-2.142163,-2.184183,-2.111332,-2.077166,-2.125910,0.587184,-0.650028,...,-0.049936,0.645604,-1.592823,0.546749,0.078090,-0.237026,-0.837124,-0.175958,0.061469,-2.056965
52,-2.098709,-0.344334,-2.120115,-2.139050,-2.178879,-2.117365,-2.077906,-2.121902,0.551176,-0.605426,...,-0.049715,0.155241,0.435721,0.581028,6.329870,-1.246710,0.090208,-0.326489,0.989154,-2.098709
53,-2.101239,0.232217,-2.118136,-2.136693,-2.174291,-2.119588,-2.081983,-2.115752,0.513769,-0.573078,...,-0.049288,0.938548,0.039596,0.103050,0.078090,0.572390,-0.203051,-1.432887,0.008242,-2.101239
54,-2.114521,0.606201,-2.119995,-2.136444,-2.171350,-2.119997,-2.105306,-2.108239,0.467045,-0.569404,...,-0.048312,0.724147,0.148211,-0.139122,1.708155,0.405500,0.288356,0.019731,0.152717,-2.114521


In [7]:
# Predict for next 6 days using last 60 days data
sequence_length = 60
window =6

def create_dataset(dataset,target,features, lookback, window):
    X_store, y_store = [], []
    for i in range(0,len(dataset)-lookback-window,3):
        X = (dataset[features].values)[i:i+lookback]
        y = (dataset[target].values)[i+lookback:i+lookback+window]
        X_store.append(X)
        y_store.append(y)
    return torch.FloatTensor(np.array(X_store)), torch.FloatTensor(np.array(y_store))

In [8]:
# Split train test data
from sklearn.model_selection import train_test_split
# df_train, df_test = train_test_split(df, test_size=0.05)
size=int(len(df))-10
df_train=df[:size]
df_test=df[size:]
print('Total sample', len(df))
print('Train sample', len(df_train))
print('Test sample', len(df_test))

Total sample 3208
Train sample 3198
Test sample 10


In [9]:
# Train data
X_train, y_train = create_dataset(
    df_train,
    target=target,
    features=features,
    lookback=sequence_length, window=window)

indices = tf.range(start=0, limit=tf.shape(X_train)[0])
shuffled_indices = tf.random.shuffle(indices)
X_train = tf.gather(X_train, shuffled_indices)
y_train = tf.gather(y_train, shuffled_indices)

X_train=torch.FloatTensor(np.array(X_train))
y_train=torch.FloatTensor(np.array(y_train))

print(X_train.shape, y_train.shape)

torch.Size([1044, 60, 29]) torch.Size([1044, 6])


2024-03-07 19:48:04.650715: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [10]:
batch_size=10
train_loader = DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)

## QLSTM-DRC

In [11]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")
    return avg_loss

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")
    return avg_loss

In [12]:
class QLSTM(nn.Module):
    def __init__(self, 
                input_size, 
                hidden_size, 
                n_qubits=6, # number of Qubits
                n_inp_size=18, # Decide the number of input to quantum model
                n_qlayers=12, #  
                n_vrotations=3,
                batch_first=True,
                return_sequences=False, 
                return_state=False,
                backend="default.qubit"):
        super(QLSTM, self).__init__()
        self.n_inputs = input_size 
        self.hidden_size = hidden_size
        self.n_inp_size=n_inp_size
        self.concat_size = self.n_inputs + self.hidden_size
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.n_vrotations = n_vrotations
        self.backend = backend  # "default.qubit", "qiskit.basicaer", "qiskit.ibm"

        self.batch_first = batch_first
        self.return_sequences = return_sequences
        self.return_state = return_state
        
        self.wires_forget = [f"wire_forget_{i}" for i in range(self.n_qubits)]
        self.wires_input = [f"wire_input_{i}" for i in range(self.n_qubits)]
        self.wires_update = [f"wire_update_{i}" for i in range(self.n_qubits)]
        self.wires_output = [f"wire_output_{i}" for i in range(self.n_qubits)]

        self.dev_forget = qml.device(self.backend, wires=self.wires_forget, shots=None)
        self.dev_input = qml.device(self.backend, wires=self.wires_input, shots=None)
        self.dev_update = qml.device(self.backend, wires=self.wires_update, shots=None)
        self.dev_output = qml.device(self.backend, wires=self.wires_output, shots=None)
         
        # 6 qubit DRC circuit    
        def VQC(inputs, weights, wires_type): # inputs, weights, self.wires_update   
            for p1,p2,p3,p4,p5,p6 in zip(weights[:2],weights[2:4],weights[4:6],weights[6:8],weights[8:10],weights[10:12]):
                qml.Rot(*inputs[:3], wires=wires_type[0])
                qml.Rot(*inputs[3:6], wires=wires_type[1])
                qml.Rot(*inputs[6:9], wires=wires_type[2])
                qml.Rot(*inputs[9:12], wires=wires_type[3])
                qml.Rot(*inputs[12:15], wires=wires_type[4])
                qml.Rot(*inputs[15:18], wires=wires_type[5])
                qml.Rot(*p1, wires=wires_type[0])
                qml.Rot(*p2, wires=wires_type[1])
                qml.Rot(*p3, wires=wires_type[2])
                qml.Rot(*p4, wires=wires_type[3])
                qml.Rot(*p5, wires=wires_type[4])
                qml.Rot(*p6, wires=wires_type[5])
                qml.CNOT(wires=[wires_type[0], wires_type[1]])
                qml.CNOT(wires=[wires_type[1], wires_type[2]])
                qml.CNOT(wires=[wires_type[2], wires_type[3]]) 
                qml.CNOT(wires=[wires_type[3], wires_type[4]])
                qml.CNOT(wires=[wires_type[4], wires_type[5]])
               
        def _circuit_forget(inputs, weights):
            VQC(inputs, weights, self.wires_forget)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_forget]
        self.qlayer_forget = qml.QNode(_circuit_forget, self.dev_forget, interface="torch", diff_method='backprop')

        def _circuit_input(inputs, weights):
            VQC(inputs, weights, self.wires_input)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_input]
        self.qlayer_input = qml.QNode(_circuit_input, self.dev_input, interface="torch", diff_method='backprop')

        def _circuit_update(inputs, weights):
            VQC(inputs, weights, self.wires_update)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_update]
        self.qlayer_update = qml.QNode(_circuit_update, self.dev_update, interface="torch", diff_method='backprop')

        def _circuit_output(inputs, weights):
            VQC(inputs, weights, self.wires_output)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_output]
        self.qlayer_output = qml.QNode(_circuit_output, self.dev_output, interface="torch", diff_method='backprop')

        weight_shapes = {"weights": (self.n_qlayers, self.n_vrotations)}
        print(f"weight_shapes = (n_qlayers, n_vrotations) = ({self.n_qlayers}, {self.n_vrotations})")

        self.clayer_in = torch.nn.Linear(self.concat_size, self.n_inp_size)
        self.VQC = {
            'forget': qml.qnn.TorchLayer(self.qlayer_forget, weight_shapes),
            'input': qml.qnn.TorchLayer(self.qlayer_input, weight_shapes),
            'update': qml.qnn.TorchLayer(self.qlayer_update, weight_shapes),
            'output': qml.qnn.TorchLayer(self.qlayer_output, weight_shapes)
        }
        self.clayer_out = torch.nn.Linear(self.n_qubits, self.hidden_size)

    def forward(self, x, init_states=None):
        '''
        x.shape is (batch_size, seq_length, feature_size)
        recurrent_activation -> sigmoid
        activation -> tanh
        '''
        if self.batch_first is True:
            batch_size, seq_length, features_size = x.size()
        else:
            seq_length, batch_size, features_size = x.size()

        hidden_seq = []
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size)  # hidden state (output)
            c_t = torch.zeros(batch_size, self.hidden_size)  # cell state
        else:
            # for now we ignore the fact that in PyTorch you can stack multiple RNNs
            # so we take only the first elements of the init_states tuple init_states[0][0], init_states[1][0]
            h_t, c_t = init_states
            h_t = h_t[0]
            c_t = c_t[0]

        for t in range(seq_length):
            # get features from the t-th element in seq, for all entries in the batch
            x_t = x[:, t, :]
            
            # Concatenate input and hidden state
            v_t = torch.cat((h_t, x_t), dim=1)

            # match qubit dimension
            y_t = self.clayer_in(v_t)

            f_t = torch.sigmoid(self.clayer_out(self.VQC['forget'](y_t)))  # forget block
            i_t = torch.sigmoid(self.clayer_out(self.VQC['input'](y_t)))  # input block
            g_t = torch.tanh(self.clayer_out(self.VQC['update'](y_t)))  # update block
            o_t = torch.sigmoid(self.clayer_out(self.VQC['output'](y_t))) # output block

            c_t = (f_t * c_t) + (i_t * g_t)
            h_t = o_t * torch.tanh(c_t)

            hidden_seq.append(h_t.unsqueeze(0))
        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)

In [13]:
class QShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units, n_qubits=6, n_qlayers=12): # n_qlayers decide the weights
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 2 # Number of QLSTM layer

        self.lstm = QLSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            n_qubits = n_qubits,
            n_qlayers= n_qlayers
        )

        self.linear_1 = nn.Linear(in_features=self.hidden_units, out_features=1)
        self.linear_2 = nn.Linear(in_features=60, out_features=6)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        out_l1, (hn, _) = self.lstm(x, (h0, c0))
        out_l2 = self.linear_1(out_l1)
        out = self.linear_2(torch.squeeze(out_l2))
        return out

In [14]:
learning_rate = 0.05
num_hidden_units = 16
 
Qmodel = QShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units, n_qubits=6) # Number of Qubits
loss_function = nn.MSELoss()
optimizer = torch.optim.Adagrad(Qmodel.parameters(), lr=learning_rate)

weight_shapes = (n_qlayers, n_vrotations) = (12, 3)


In [ ]:
quantum_loss_train = []
quantum_loss_test = []
num_epoch=5
for ix_epoch in range(num_epoch):
    print(f"Epoch {ix_epoch}\n---------")
    start = time.time()
    train_loss = train_model(train_loader, Qmodel, loss_function, optimizer=optimizer)
    end = time.time()
    print("Execution time", end - start)
    quantum_loss_train.append(train_loss)

Epoch 0
---------
Train loss: 0.24071206104542528
Execution time 11385.806482076645
Epoch 1
---------


In [16]:
quantum_loss_train

[0.24071206104542528,
 0.0738448812670651,
 0.056562427023337,
 0.04531906358010712,
 0.042639000350165934]

# Predict for 8th to 15 march

In [18]:
last_60 = pd.read_csv('last_60_days_input.csv')
last_60 = last_60.drop([ 'Unnamed: 0'], axis=1)
# Remove few redundant data
last_60=last_60.drop(columns=['Date'])
last_60=last_60.drop(columns=['CUMLOGRET_1'])
last_60=last_60.drop(columns=['Gold in USD volume'])
last_60=last_60.drop(columns=['Open'])
last_60=last_60.drop(columns=['High'])
last_60=last_60.drop(columns=['Low'])

In [25]:
for i in features:
    last_60[i]=(last_60[i]-mean[i])/stdev[i]

In [26]:
X_last_60=np.array(last_60[features])

In [27]:
y_8_to_15 = Qmodel(torch.FloatTensor([X_last_60]))
y_8_to_15 = y_8_to_15.detach().numpy()* target_stdev + target_mean

/tmp/ipykernel_21066/1693403120.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  y_8_to_15 = Qmodel(torch.FloatTensor([X_last_60]))


In [28]:
y_8_to_15

array([7662.464 , 7662.884 , 7645.0625, 7640.8667, 7660.0586, 7624.8257],
      dtype=float32)

In [1]:
# total_params_Q = sum(p.numel() for p in Qmodel.parameters() if p.requires_grad)
# print("No. of parameters for QLSTM: ", total_params_Q)
# Number of parameter is 1323